In [1]:
import numpy as np
import video_sync as vs
import utm
import math
import random

In [2]:
def extract_gps_data(jsonfile):
    data = vs.object_init(jsonfile)
    lat = vs.extract_attribute(data,"latitude")
    lon = vs.extract_attribute(data,"longitude")
    acc = vs.extract_attribute(data,"accuracy")
    time = np.array(vs.extract_attribute(data,"timestamp"))/1000
    frame = vs.extract_attribute(data, "frame")

    data_final = np.column_stack((time,frame,lat,lon,acc))
    return data_final

def extract_slam_data(trajectory):
    f = open(trajectory, "r")
    runs = []
    data = []
    for i,line in enumerate(f):
        if line == "################################################## \n":
            if len(data) != 0:
                runs.append(np.array(data,dtype='float'))
                data = []
        else:
            l = line.split(" ")
            data.append(l[0:4])
    runs.append(np.array(data,dtype='float'))
    return runs

In [3]:
def time_sync(gps_data, slam_runs):
    t1 = gps_data[0][0]
    t2 = slam_runs[0][0][0]

    offset = t2 - t1
    for slam_data in slam_runs:
        slam_data[:,0] = slam_data[:,0] - offset

    return slam_runs


In [4]:
def sample_three(gps_data, slam_data):
    #Find all the reasonably accurate GPS points (between 0 and 5 meters)    
    start = np.searchsorted(gps_data[:,0],slam_data[0][0],'left')
    end = np.searchsorted(gps_data[:,0], slam_data[-1][0],'left')
    
    good_points = np.where(np.logical_and(gps_data[start:end,4] > -1, gps_data[start:end,4]<5))
    indices = np.array(random.sample(good_points[0],3)) + start
    corsp_slam = []
    for t in gps_data[indices][:,0]:
        idx = np.searchsorted(slam_data[:,0],t, 'left')
        corsp_slam.append(idx)

    return gps_data[indices][:,[0,2,3]], slam_data[corsp_slam]

In [5]:
def scale_and_combine(gps_data, slam_data):
    gps_samples,slam_samples =  sample_three(gps_data, slam_data)
    utm_samples = []
    for i in xrange(0,3):
        u = list(utm.from_latlon(gps_samples[i][1], gps_samples[i][2]))[:2]
        u.append(0)
        utm_samples.append(u)

    A = recover_homogenous_affine_transformation(np.array(slam_samples[:,1:4]),np.array(utm_samples))
    
    slam_converted = np.zeros((len(slam_data),3))
    
    for i,obv in enumerate(slam_data):
        slam_converted[i][0] = obv[0]
        ### One and Three are the two axis aligned to the plane.
        utm_conv = transform_pt(obv[1:],A)
        gps_conv = utm.to_latlon(utm_conv[0], utm_conv[1],18, 'T')
        print gps_conv
        slam_converted[i][1:3] = gps_conv

    return slam_converted

In [6]:
def slam2gps(jsonfile, frametrajectory):   
    gps_data = extract_gps_data(jsonfile)
    slam_data = extract_slam_data(frametrajectory)
    slam_data = time_sync(slam_data, gps_data)

    location_data_final = scale_and_combine(gps_data, slam_data)

    return location_data_final

In [7]:
def recover_homogenous_affine_transformation(p, p_prime):
    '''
    Find the unique homogeneous affine transformation that
    maps a set of 3 points to another set of 3 points in 3D
    space:

        p_prime == np.dot(p, R) + t

    where `R` is an unknown rotation matrix, `t` is an unknown
    translation vector, and `p` and `p_prime` are the original
    and transformed set of points stored as row vectors:

        p       = np.array((p1,       p2,       p3))
        p_prime = np.array((p1_prime, p2_prime, p3_prime))

    The result of this function is an augmented 4-by-4
    matrix `A` that represents this affine transformation:

        np.column_stack((p_prime, (1, 1, 1))) == \
            np.dot(np.column_stack((p, (1, 1, 1))), A)

    Source: https://math.stackexchange.com/a/222170 (robjohn)
    '''

    # construct intermediate matrix
    Q       = p[1:]       - p[0]
    Q_prime = p_prime[1:] - p_prime[0]

    # calculate rotation matrix
    R = np.dot(np.linalg.inv(np.row_stack((Q, np.cross(*Q)))),
               np.row_stack((Q_prime, np.cross(*Q_prime))))

    # calculate translation vector
    t = p_prime[0] - np.dot(p[0], R)

    # calculate affine transformation matrix
    return np.column_stack((np.row_stack((R, t)),
                            (0, 0, 0, 1)))



def transform_pt(point, trans_mat):
    a  = np.array([point[0], point[1], point[2], 1])
    ap = np.dot(a, trans_mat)[:3]
    return [ap[0], ap[1], ap[2]]



In [8]:
gps_data = extract_gps_data("/home/carmera/Documents/data/json/cds-1145ca086c7f4f26-20170608-0742.json")
slam_runs = extract_slam_data("/home/carmera/KeyFrame-test.txt")
slam_runs = time_sync(gps_data, slam_runs)

In [37]:
final_data = scale_and_combine(gps_data,slam_runs[1])

LinAlgError: Singular matrix

In [34]:
final_data = scale_and_combine(gps_data,slam_runs[1])

(40.717834524817455, -73.94338544860342)
(40.71791464741644, -73.94339087835698)
(40.7178724924942, -73.94338394654098)
(40.71775522390141, -73.94336861142997)
(40.71761026001542, -73.94335232089949)
(40.71728297961999, -73.94331730636506)
(40.716994278307254, -73.94328602418103)
(40.71702799303573, -73.94328643785578)
(40.7168823106688, -73.94326894642296)
(40.71680268200977, -73.94325835963502)
(40.716674018735915, -73.94324301900672)
(40.71637187740044, -73.94321095680054)
(40.716434766691606, -73.94321407277883)
(40.71662956743555, -73.94322871807935)
(40.71728425635202, -73.94328826825823)
(40.717823362821214, -73.94333598414178)
(40.71838810098952, -73.94338735861648)
(40.71862820405965, -73.94340801215027)
(40.71870517813106, -73.94341290164384)
(40.71863124934946, -73.94340323746339)
(40.718584322543116, -73.9433961947799)
(40.71860012363449, -73.94339520852924)
(40.71848085725317, -73.94338115799758)
(40.71840248128803, -73.94336982713995)
(40.71841425281071, -73.9433697060075

In [15]:
gps_data[5:,[2,3]]

array([[ 40.7185785, -73.943387 ],
       [ 40.7185785, -73.943387 ],
       [ 40.7185785, -73.943387 ],
       [ 40.7183314, -73.9433534],
       [ 40.7181109, -73.9433244],
       [ 40.7178732, -73.943304 ],
       [ 40.7178732, -73.943304 ],
       [ 40.7176632, -73.9432501],
       [ 40.7174403, -73.9432112],
       [ 40.7174403, -73.9432112],
       [ 40.7172049, -73.9431733],
       [ 40.7172049, -73.9431733],
       [ 40.7169784, -73.9431383],
       [ 40.7169784, -73.9431383],
       [ 40.7166861, -73.94308  ],
       [ 40.7166861, -73.94308  ],
       [ 40.7165206, -73.9430668],
       [ 40.7165206, -73.9430668],
       [ 40.7165206, -73.9430668],
       [ 40.7165206, -73.9430668],
       [ 40.7165206, -73.9430668],
       [ 40.7164179, -73.9430316],
       [ 40.7164179, -73.9430316],
       [ 40.7164179, -73.9430316],
       [ 40.7164179, -73.9430316],
       [ 40.7164179, -73.9430316],
       [ 40.7164177, -73.9430575],
       [ 40.7164177, -73.9430575],
       [ 40.7164177,

array([[ -1.06e+02,   7.35e+02,   1.69e+04,   0.00e+00],
       [ -4.12e+03,   2.86e+04,  -4.49e+02,   0.00e+00],
       [ -1.77e+02,   1.22e+03,   3.27e+02,   0.00e+00],
       [  5.89e+05,   4.51e+06,  -1.51e+02,   1.00e+00]])

In [265]:
np.set_printoptions(precision=8)